# TFT Match History Crawler
Lấy lịch sử đấu Teamfight Tactics từ tên người chơi sử dụng Riot API

In [ ]:
# ========================================
# CELL 1: CONFIG
# ========================================

# Riot API Key - Lấy từ https://developer.riotgames.com/
RIOT_API_KEY = "YOUR_RIOT_API_KEY_HERE"

# Thông tin người chơi
GAME_NAME = "Hoàng Đế Shurima"  # Tên game (phần trước #)
TAG_LINE = "2911"            # Tag (phần sau #, ví dụ: VN2, EUW, NA1)

# Region settings
# Platform routing: vn2, euw1, na1, kr, jp1, etc.
PLATFORM = "vn2"

# Regional routing: americas, asia, europe, sea
REGION = "asia"

# Số lượng trận đấu muốn lấy (tối đa 200)
MATCH_COUNT = 20

# API URLs
ACCOUNT_API_URL = f"https://{REGION}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{GAME_NAME}/{TAG_LINE}"
MATCH_LIST_BASE_URL = f"https://sea.api.riotgames.com/tft/match/v1/matches/by-puuid"
MATCH_DETAIL_BASE_URL = f"https://sea.api.riotgames.com/tft/match/v1/matches"

print("✅ Config loaded successfully!")
print(f"📍 Player: {GAME_NAME}#{TAG_LINE}")
print(f"🌍 Region: {REGION} | Platform: {PLATFORM}")
print(f"📊 Will fetch {MATCH_COUNT} matches")

✅ Config loaded successfully!
📍 Player: Hoàng Đế Shurima#2911
🌍 Region: asia | Platform: vn2
📊 Will fetch 20 matches


In [ ]:
# ========================================
# CELL 2: IMPORTS & HELPER FUNCTIONS
# ========================================

import requests
import time
import json
from datetime import datetime
import pandas as pd

def make_api_request(url, params=None):
    """Gửi request đến Riot API với xử lý rate limit"""
    headers = {
        "X-Riot-Token": RIOT_API_KEY
    }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:
            # Rate limited - wait and retry
            retry_after = int(response.headers.get('Retry-After', 10))
            print(f"⏳ Rate limited. Waiting {retry_after} seconds...")
            time.sleep(retry_after)
            return make_api_request(url, params)
        elif response.status_code == 404:
            print(f"❌ Not found: {url}")
            return None
        elif response.status_code == 403:
            print("❌ API Key invalid or expired. Please check your API_KEY.")
            return None
        else:
            print(f"❌ Error {response.status_code}: {response.text}")
            return None
    except Exception as e:
        print(f"❌ Request failed: {e}")
        return None

def format_timestamp(timestamp_ms):
    """Chuyển đổi timestamp (ms) sang datetime readable"""
    return datetime.fromtimestamp(timestamp_ms / 1000).strftime('%Y-%m-%d %H:%M:%S')

print("✅ Helper functions loaded!")

✅ Helper functions loaded!


In [6]:
# ========================================
# CELL 3: GET PLAYER PUUID
# ========================================

print(f"🔍 Fetching PUUID for {GAME_NAME}#{TAG_LINE}...")

account_data = make_api_request(ACCOUNT_API_URL)

if account_data:
    puuid = account_data['puuid']
    print(f"✅ Found player!")
    print(f"   Game Name: {account_data.get('gameName', 'N/A')}")
    print(f"   Tag Line: {account_data.get('tagLine', 'N/A')}")
    print(f"   PUUID: {puuid[:20]}...")
else:
    puuid = None
    print("❌ Failed to get player PUUID. Check player name and tag.")

🔍 Fetching PUUID for Hoàng Đế Shurima#2911...
✅ Found player!
   Game Name: Hoàng Đế Shurima
   Tag Line: 2911
   PUUID: 2Ou-PECRAzzALWJKtg5v...


In [13]:
# ========================================
# CELL 4: GET MATCH HISTORY
# ========================================

if puuid:
    print(f"📋 Fetching {MATCH_COUNT} recent matches...")
    
    match_list_url = f"{MATCH_LIST_BASE_URL}/{puuid}/ids"
    params = {
        "count": MATCH_COUNT
    }
    
    match_ids = make_api_request(match_list_url, params)
    
    if match_ids:
        print(f"✅ Found {len(match_ids)} matches!")
        print("\n📝 Match IDs:")
        for i, match_id in enumerate(match_ids[:5], 1):
            print(f"   {i}. {match_id}")
        if len(match_ids) > 5:
            print(f"   ... and {len(match_ids) - 5} more")
    else:
        match_ids = []
        print("❌ Failed to get match history.")
else:
    match_ids = []
    print("⚠️ Skipping - PUUID not available.")

📋 Fetching 20 recent matches...
✅ Found 20 matches!

📝 Match IDs:
   1. VN2_1226241991
   2. VN2_1226080788
   3. VN2_1224936601
   4. VN2_1224815517
   5. VN2_1224364107
   ... and 15 more


In [14]:
# ========================================
# CELL 5: GET MATCH DETAILS
# ========================================

matches_data = []

if match_ids:
    print(f"📊 Fetching details for {len(match_ids)} matches...")
    print("(This may take a while due to API rate limits)\n")
    
    for i, match_id in enumerate(match_ids, 1):
        print(f"⏳ [{i}/{len(match_ids)}] Fetching {match_id}...", end=" ")
        
        match_url = f"{MATCH_DETAIL_BASE_URL}/{match_id}"
        match_detail = make_api_request(match_url)
        
        if match_detail:
            matches_data.append(match_detail)
            print("✅")
        else:
            print("❌")
        
        # Small delay to avoid rate limiting
        time.sleep(0.5)
    
    print(f"\n✅ Successfully fetched {len(matches_data)} match details!")
else:
    print("⚠️ No matches to fetch.")

📊 Fetching details for 20 matches...
(This may take a while due to API rate limits)

⏳ [1/20] Fetching VN2_1226241991... ✅
⏳ [2/20] Fetching VN2_1226080788... ✅
⏳ [3/20] Fetching VN2_1224936601... ✅
⏳ [4/20] Fetching VN2_1224815517... ✅
⏳ [5/20] Fetching VN2_1224364107... ✅
⏳ [6/20] Fetching VN2_1224336208... ✅
⏳ [7/20] Fetching VN2_1224317477... ✅
⏳ [8/20] Fetching VN2_1224297973... ✅
⏳ [9/20] Fetching VN2_1219398294... ✅
⏳ [10/20] Fetching VN2_1219321060... ✅
⏳ [11/20] Fetching VN2_1217943125... ✅
⏳ [12/20] Fetching VN2_1217906219... ✅
⏳ [13/20] Fetching VN2_1217883002... ✅
⏳ [14/20] Fetching VN2_1216601396... ✅
⏳ [15/20] Fetching VN2_1215658429... ✅
⏳ [16/20] Fetching VN2_1212749007... ✅
⏳ [17/20] Fetching VN2_1212668560... ✅
⏳ [18/20] Fetching VN2_1212652117... ✅
⏳ [19/20] Fetching VN2_1212505615... ✅
⏳ [20/20] Fetching VN2_1212454459... ✅

✅ Successfully fetched 20 match details!


In [15]:
# ========================================
# CELL 6: PROCESS & DISPLAY RESULTS
# ========================================

def extract_player_stats(match_data, target_puuid):
    """Trích xuất thông tin của người chơi từ match data"""
    info = match_data.get('info', {})
    metadata = match_data.get('metadata', {})
    
    # Tìm participant của người chơi
    participants = info.get('participants', [])
    player_data = None
    
    for participant in participants:
        if participant.get('puuid') == target_puuid:
            player_data = participant
            break
    
    if not player_data:
        return None
    
    # Lấy traits (synergies)
    traits = player_data.get('traits', [])
    active_traits = [t['name'] for t in traits if t.get('tier_current', 0) > 0]
    
    # Lấy units (champions)
    units = player_data.get('units', [])
    unit_names = [u.get('character_id', 'Unknown') for u in units]
    
    return {
        'match_id': metadata.get('match_id', 'N/A'),
        'game_datetime': format_timestamp(info.get('game_datetime', 0)),
        'game_length': round(info.get('game_length', 0) / 60, 1),  # Convert to minutes
        'game_version': info.get('game_version', 'N/A'),
        'tft_set_number': info.get('tft_set_number', 'N/A'),
        'placement': player_data.get('placement', 'N/A'),
        'level': player_data.get('level', 'N/A'),
        'gold_left': player_data.get('gold_left', 0),
        'players_eliminated': player_data.get('players_eliminated', 0),
        'total_damage_to_players': player_data.get('total_damage_to_players', 0),
        'active_traits': ', '.join(active_traits[:5]),  # Top 5 traits
        'units': ', '.join(unit_names),
        'num_units': len(units)
    }

# Process all matches
if matches_data and puuid:
    results = []
    for match in matches_data:
        stats = extract_player_stats(match, puuid)
        if stats:
            results.append(stats)
    
    # Create DataFrame
    df = pd.DataFrame(results)
    
    print(f"\n📊 Match History for {GAME_NAME}#{TAG_LINE}")
    print("=" * 60)
    
    # Display summary
    if len(df) > 0:
        avg_placement = df['placement'].mean()
        top4_rate = (df['placement'] <= 4).sum() / len(df) * 100
        win_rate = (df['placement'] == 1).sum() / len(df) * 100
        
        print(f"\n📈 Summary Statistics:")
        print(f"   Total Matches: {len(df)}")
        print(f"   Avg Placement: {avg_placement:.2f}")
        print(f"   Top 4 Rate: {top4_rate:.1f}%")
        print(f"   Win Rate: {win_rate:.1f}%")
    
    # Display DataFrame
    display_cols = ['game_datetime', 'placement', 'level', 'game_length', 'active_traits']
    print("\n📋 Recent Matches:")
    display(df[display_cols].head(10))
else:
    print("⚠️ No match data to process.")
    df = pd.DataFrame()


📊 Match History for Hoàng Đế Shurima#2911

📈 Summary Statistics:
   Total Matches: 20
   Avg Placement: 3.40
   Top 4 Rate: 65.0%
   Win Rate: 25.0%

📋 Recent Matches:


,game_datetime,placement,level,game_length,active_traits
0,2026-01-20 13:48:42,1,8,36.6,"TFT16_Defender, TFT16_Juggernaut, TFT16_Kindre..."
1,2026-01-20 11:33:38,3,9,35.4,"TFT16_Bilgewater, TFT16_Brawler, TFT16_Glutton..."
2,2026-01-19 16:43:27,3,9,32.4,"TFT16_Bilgewater, TFT16_Brawler, TFT16_Harvest..."
3,2026-01-19 15:15:16,4,9,36.7,"TFT16_Brawler, TFT16_Defender, TFT16_Ionia, TF..."
4,2026-01-19 08:04:15,1,8,32.2,"TFT16_Defender, TFT16_Emperor, TFT16_Magus, TF..."
5,2026-01-19 06:56:52,6,8,34.8,"TFT16_Defender, TFT16_Demacia, TFT16_Ionia, TF..."
6,2026-01-19 05:59:48,3,9,35.4,"TFT16_Defender, TFT16_Demacia, TFT16_Heroic, T..."
7,2026-01-19 05:09:21,3,10,37.5,"TFT16_Chronokeeper, TFT16_Defender, TFT16_Dema..."
8,2026-01-16 01:57:52,2,10,34.9,"TFT16_Bilgewater, TFT16_Brawler, TFT16_Glutton..."
9,2026-01-16 00:38:48,1,9,34.2,"TFT16_Defender, TFT16_Demacia, TFT16_Heroic, T..."


In [16]:
# ========================================
# CELL 7: SAVE DATA
# ========================================

if len(df) > 0:
    # Save to CSV
    filename = f"tft_history_{GAME_NAME}_{TAG_LINE}.csv"
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"✅ Data saved to {filename}")
    
    # Save raw JSON for detailed analysis
    json_filename = f"tft_history_{GAME_NAME}_{TAG_LINE}_raw.json"
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(matches_data, f, ensure_ascii=False, indent=2)
    print(f"✅ Raw data saved to {json_filename}")
else:
    print("⚠️ No data to save.")

✅ Data saved to tft_history_Hoàng Đế Shurima_2911.csv
✅ Raw data saved to tft_history_Hoàng Đế Shurima_2911_raw.json


In [17]:
# ========================================
# CELL 8: EXPORT FULL DATA
# ========================================

# Display full DataFrame with all columns
if len(df) > 0:
    print("📋 Full Match Data:\n")
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    display(df)
else:
    print("⚠️ No data available.")

📋 Full Match Data:



,match_id,game_datetime,game_length,game_version,tft_set_number,placement,level,gold_left,players_eliminated,total_damage_to_players,active_traits,units,num_units
0,VN2_1226241991,2026-01-20 13:48:42,36.6,Linux Version 16.1.737.4870 (Jan 09 2026/19:00...,16,1,8,2,2,159,"TFT16_Defender, TFT16_Juggernaut, TFT16_Kindre...","TFT16_Neeko, TFT16_Vi, TFT16_Ekko, TFT16_Serap...",8
1,VN2_1226080788,2026-01-20 11:33:38,35.4,Linux Version 16.1.737.4870 (Jan 09 2026/19:00...,16,3,9,1,1,168,"TFT16_Bilgewater, TFT16_Brawler, TFT16_Glutton...","TFT16_TwistedFate, TFT16_MissFortune, TFT16_Ta...",9
2,VN2_1224936601,2026-01-19 16:43:27,32.4,Linux Version 16.1.737.4870 (Jan 09 2026/19:00...,16,3,9,1,0,113,"TFT16_Bilgewater, TFT16_Brawler, TFT16_Harvest...","TFT16_Illaoi, TFT16_Sion, TFT16_RekSai, TFT16_...",9
3,VN2_1224815517,2026-01-19 15:15:16,36.7,Linux Version 16.1.737.4870 (Jan 09 2026/19:00...,16,4,9,4,1,99,"TFT16_Brawler, TFT16_Defender, TFT16_Ionia, TF...","TFT16_Caitlyn, TFT16_Vi, TFT16_Kennen, TFT16_D...",9
4,VN2_1224364107,2026-01-19 08:04:15,32.2,Linux Version 16.1.737.4870 (Jan 09 2026/19:00...,16,1,8,3,3,137,"TFT16_Defender, TFT16_Emperor, TFT16_Magus, TF...","TFT16_Neeko, TFT16_Vi, TFT16_Seraphine, TFT16_...",8
5,VN2_1224336208,2026-01-19 06:56:52,34.8,Linux Version 16.1.737.4870 (Jan 09 2026/19:00...,16,6,8,1,0,57,"TFT16_Defender, TFT16_Demacia, TFT16_Ionia, TF...","TFT16_JarvanIV, TFT16_Sona, TFT16_XinZhao, TFT...",8
6,VN2_1224317477,2026-01-19 05:59:48,35.4,Linux Version 16.1.737.4870 (Jan 09 2026/19:00...,16,3,9,0,1,135,"TFT16_Defender, TFT16_Demacia, TFT16_Heroic, T...","TFT16_JarvanIV, TFT16_Sona, TFT16_XinZhao, TFT...",10
7,VN2_1224297973,2026-01-19 05:09:21,37.5,Linux Version 16.1.737.4870 (Jan 09 2026/19:00...,16,3,10,3,1,94,"TFT16_Chronokeeper, TFT16_Defender, TFT16_Dema...","TFT16_JarvanIV, TFT16_Sona, TFT16_Poppy, TFT16...",11
8,VN2_1219398294,2026-01-16 01:57:52,34.9,Linux Version 16.1.737.4870 (Jan 09 2026/19:00...,16,2,10,1,1,141,"TFT16_Bilgewater, TFT16_Brawler, TFT16_Glutton...","TFT16_Graves, TFT16_Poppy, TFT16_Kobuko, TFT16...",10
9,VN2_1219321060,2026-01-16 00:38:48,34.2,Linux Version 16.1.737.4870 (Jan 09 2026/19:00...,16,1,9,57,4,179,"TFT16_Defender, TFT16_Demacia, TFT16_Heroic, T...","TFT16_JarvanIV, TFT16_Sona, TFT16_Taric, TFT16...",10
